In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import dataset
import predict

In [2]:
data = pd.read_csv('data/EPL_processed_results.csv', index_col=0)
X_train, y_train, X_val, y_val, X_test, y_test = dataset.generate_train_val_test_sets(data)

# Load a trained model

In [3]:
predictor = predict.Predictor()
predictor.load_saved_model('model')

2022-04-03 15:19:36.037025: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Evaluate on test set

In [4]:
loss, acc = predictor.evaluate(X_test, y_test)

9/9 [==============================] - 1s 3ms/step - loss: 0.9875 - accuracy: 0.5324


The trained model has an accuracy of 0.5324 on the test set. Let's compare this with a simple benchmark: guess a home win for every single game.

In [5]:
from sklearn.metrics import accuracy_score
y_pred_naive = [[1, 0, 0]] * len(y_test)
acc_naive = accuracy_score(y_test, y_pred_naive)

print("Accuracy of naive benchmark: %.4f" % acc_naive)

Accuracy of naive benchmark: 0.4209


Our model achieves a better accuracy than the naive benchmark.

# Predict on a new sample

To make a prediction on a new match, we need to format it the same way as the rest of the dataset, i.e. as a dataframe with the required columns (see predict.py for a list of required columns). Here is an example.

In [6]:
sample = data.iloc[3556].to_frame().transpose()
print(sample)

       Season        Date HomeTeam     AwayTeam FTR HomeGoals AwayGoals  \
3556  2021-22  2021-08-14  Everton  Southampton   H         3         1   

     StandingDiff HomeWins AwayWins  ... AvgHomeShots AvgAwayShots  \
3556           -5     20.0      0.0  ...         15.8         12.8   

     AvgHomeShotsOnTarget AvgAwayShotsOnTarget AvgHomeCorners AvgAwayCorners  \
3556                  5.4                  5.2            6.0            5.4   

     AvgHomeGoalsConceded AvgAwayGoalsConceded AvgHomeShotsConceded  \
3556                  1.2                  3.0                 11.4   

     AvgAwayShotsConceded  
3556                 14.4  

[1 rows x 24 columns]


In [7]:
predictor.predict(sample)

Predicted outcome: 0.5233 home win - 0.2332 draw - 0.2435 away win
